In [9]:
#\\Standard libraries
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import time
import math
import sys
import pickle as pkl
import os
import sys
import yaml

sys.path.append(os.path.dirname(os.getcwd()))


#\\Own libraries
from data.sample_generator import *
from utils.util import *
from model.langevin import *
from runners.train import *
from pathlib import Path


In [10]:
######################
###  Load parameters of the system ###
######################
dirPath = os.path.dirname(os.getcwd())
with open(dirPath + '/config_training.yml', 'r') as f:
    aux = yaml.load(f,  Loader=yaml.FullLoader)
config = dict2namespace(aux)


######################
###  General setup ###
######################
SEED = 123
torch.manual_seed(SEED)
useGPU = True # If true, and GPU is available, use it.
loadChannelInput = True
loadModel = True

#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda'
else:
    device = 'cpu'

In [11]:
train_batch_size = 500
test_batch_size = 500
train_iter = 16000
learning_rate = 1e-3

device = 'cuda:0'
generator = sample_generator(test_batch_size, config.mod_n, config.NR)
#Create noise vector for Langevin
sigma_gaussian = np.exp(np.linspace(np.log(config.sigma_0), np.log(config.sigma_L),config.n_sigma_gaussian))

H_test, y, x, j_indices, noise_sigma = generator.give_batch_data(config.NT, snr_db_min=config.SNR_dBs[config.NT][0], snr_db_max=config.SNR_dBs[config.NT][-1], batch_size=test_batch_size, correlated_flag=config.corr_flag, rho=config.rho)        

model = Langevin_unfolded(config.NT, config.NR, sigma_gaussian, generator, config.n_sample_init, config.step_size, device='cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
if loadModel:
    model.load_state_dict(torch.load('model_learning_64x32_16qam_3lr.pth'))
else:
    train(H_test, model ,optimizer, generator, config, train_batch_size, test_batch_size, train_iter, device)

In [5]:
n_traj = 20
SER_uLang = model_eval(model, generator, config, test_batch_size, n_traj, device)


16.0
3.3195691108703613
3.3364036083221436
3.3032760620117188
3.3029394149780273
3.3041510581970215
3.304988384246826
3.3049192428588867
3.3042783737182617
3.308452844619751
3.2958648204803467


In [ ]:
# langtrue = [0.0686, 0.0305, 0.011, 0.003 , 0.001 ,0.00018, 6e-5]
langtrue = [0.098, 0.062, 0.033, 0.022 , 0.01 , 0.0035, 0.0016]
langtrue = [0.076, 0.036, 0.016, 0.0052,0.0014,0.00033]
plt.semilogy(config.SNR_dBs[config.NT], SER_uLang, label= 'Langevin', marker = '*')
plt.grid(True, which="both")
plt.legend(loc = 1, fontsize=15)
plt.xlabel('SNR', fontsize=15)
plt.ylabel('SER', fontsize=15)
plt.tick_params(axis='both' , labelsize=15)
plt.tight_layout()

In [ ]:
config.SNR_dBs[config.NT]

In [ ]:
config.SNR_dBs[config.NT].shape[0]